In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体 - 全部使用SimHei
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.unicode_minus'] = False

# 读取数据
file_path = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\02.区县统计数据验证\四个图层在区县的均值_2021年县域统计数据_验证.csv'
df = pd.read_csv(file_path, encoding='utf-8')

print(f"数据形状: {df.shape}")
print(f"省份数量: {df['省份'].nunique()}")

# 定义四个栅格变量
raster_vars = ['F', 'F_NF', 'NF_F', 'NF']

# 定义统计指标
exclude_cols = ['区县代码', '年份', '区县', '省份', '城市'] + raster_vars
stat_vars = [col for col in df.columns if col not in exclude_cols and pd.api.types.is_numeric_dtype(df[col])]

print(f"统计指标数量: {len(stat_vars)}")

# 改进的计算函数：计算R²
def calculate_r2(x, y):
    """计算R²，处理缺失值"""
    mask = ~(np.isnan(x) | np.isnan(y) | np.isinf(x) | np.isinf(y))
    if mask.sum() < 3:
        return np.nan
    
    x_clean = x[mask]
    y_clean = y[mask]
    
    if np.var(x_clean) == 0 or np.var(y_clean) == 0:
        return np.nan
    
    try:
        r = np.corrcoef(x_clean, y_clean)[0, 1]
        r2 = r ** 2
        return r2
    except:
        return np.nan

# 存储结果
results_list = []

print("\n开始计算...")
for i, province in enumerate(df['省份'].unique(), 1):
    province_data = df[df['省份'] == province].copy()
    n_counties = len(province_data)
    
    if n_counties < 3:
        continue
    
    print(f"{i}/31 - {province} (n={n_counties})")
    
    for raster_var in raster_vars:
        raster_values = province_data[raster_var].values
        
        for stat_var in stat_vars:
            stat_values = province_data[stat_var].values
            r2 = calculate_r2(raster_values, stat_values)
            
            if not np.isnan(r2):
                results_list.append({
                    '省份': province,
                    '区县数': n_counties,
                    '栅格变量': raster_var,
                    '统计指标': stat_var,
                    'R2': r2
                })

results_df = pd.DataFrame(results_list)
print(f"\n总结果数: {len(results_df)}")

# 保存完整结果
output_file = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\分省份R2结果_完整.csv'
results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"完整结果已保存")

# ==================== 深度分析 ====================

print("\n" + "="*100)
print("1. 按R2分析 (R2 > 0.25 表示中等以上相关)")
print("="*100)

# 使用R2 > 0.25 (相当于|r| > 0.5)
moderate_corr = results_df[results_df['R2'] > 0.25].sort_values('R2', ascending=False)
print(f"\n发现 {len(moderate_corr)} 个中等以上相关的结果 (R2 > 0.25)")

moderate_corr_file = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\分省份R2_中等以上.csv'

if len(moderate_corr) > 0:
    print("\nTop 30 最强相关:")
    print(moderate_corr[['省份', '栅格变量', '统计指标', 'R2']].head(30).to_string())
    moderate_corr.to_csv(moderate_corr_file, index=False, encoding='utf-8-sig')

# 分析弱到中等相关 (0.09 < R2 < 0.25, 相当于0.3 < |r| < 0.5)
weak_moderate = results_df[(results_df['R2'] > 0.09) & (results_df['R2'] <= 0.25)]
print(f"\n发现 {len(weak_moderate)} 个弱到中等相关的结果 (0.09 < R2 < 0.25)")

print("\n" + "="*100)
print("2. 各栅格变量最相关的指标 (按R2)")
print("="*100)

for raster_var in raster_vars:
    print(f"\n{raster_var} - Top 15指标:")
    var_data = results_df[results_df['栅格变量'] == raster_var]
    
    # 按R2的平均值排序
    top_by_r2 = var_data.groupby('统计指标').agg({
        'R2': 'mean'
    }).sort_values('R2', ascending=False).head(15)
    
    print(top_by_r2.round(4))

print("\n" + "="*100)
print("3. 指标分类分析")
print("="*100)

# 将指标分类
indicator_categories = {
    '人口': ['总人口', '乡村人口', '户籍人口', '总户数', '乡村户数'],
    '就业': ['从业人员', '职工'],
    '经济总量': ['生产总值', '增加值', '总产值'],
    '收入': ['收入', '工资'],
    '农业': ['农作物', '耕地', '粮食', '棉花', '油料', '肉类', '农林牧渔', '农业机械', '农用机械'],
    '工业': ['工业企业', '工业增加值', '工业总产值'],
    '投资消费': ['投资', '消费', '储蓄', '贷款'],
    '基础设施': ['电话', '用电', '宽带'],
    '教育': ['学校', '教师', '学生'],
    '医疗': ['医院', '卫生', '床位'],
    '其他': []
}

def categorize_indicator(indicator):
    for category, keywords in indicator_categories.items():
        if category == '其他':
            continue
        for keyword in keywords:
            if keyword in indicator:
                return category
    return '其他'

results_df['指标类别'] = results_df['统计指标'].apply(categorize_indicator)

# 按类别统计平均相关性
category_stats = results_df.groupby(['栅格变量', '指标类别']).agg({
    'R2': ['mean', 'max', 'count']
}).round(4)
category_stats.columns = ['平均R2', '最大R2', '样本数']

# 修复排序 - 重置索引后排序
category_stats_reset = category_stats.reset_index()
category_stats_sorted = category_stats_reset.sort_values(['栅格变量', '平均R2'], ascending=[True, False])

print("\n各类别指标与栅格变量的R2:")
print(category_stats_sorted.to_string())

# 保存分类结果
category_file = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\指标分类R2统计.csv'
category_stats_sorted.to_csv(category_file, index=False, encoding='utf-8-sig')

print("\n" + "="*100)
print("4. 跨省份稳定的相关性 (在多个省份都显示相关)")
print("="*100)

# 降低阈值到R2 > 0.09 (相当于|r| > 0.3)
for raster_var in raster_vars:
    print(f"\n{raster_var} - 在至少5个省份R2 > 0.09的指标:")
    raster_data = results_df[results_df['栅格变量'] == raster_var]
    
    consistent = raster_data[raster_data['R2'] > 0.09].groupby('统计指标').agg({
        'R2': ['mean', 'count']
    }).round(4)
    consistent.columns = ['平均R2', '省份数']
    consistent = consistent[consistent['省份数'] >= 5].sort_values('平均R2', ascending=False)
    
    if len(consistent) > 0:
        print(consistent.head(15))
    else:
        print("  无")

print("\n" + "="*100)
print("5. 可视化分析")
print("="*100)

# 创建第一组可视化 (3个子图)
fig = plt.figure(figsize=(22, 7))

# 5.1 R2分布箱线图
ax1 = plt.subplot(1, 3, 1)
results_df.boxplot(column='R2', by='栅格变量', ax=ax1)
ax1.set_title('各栅格变量R2分布', fontsize=22, fontweight='bold')
ax1.set_xlabel('栅格变量', fontsize=22, fontweight='bold')
ax1.set_ylabel('R2', fontsize=22, fontweight='bold')
ax1.tick_params(axis='both', labelsize=18, width=1.5)
for label in ax1.get_xticklabels() + ax1.get_yticklabels():
    label.set_fontweight('bold')
plt.suptitle('')

# 5.2 各类别指标的平均R2
ax2 = plt.subplot(1, 3, 2)
category_pivot = results_df.pivot_table(values='R2', index='指标类别', columns='栅格变量', aggfunc='mean')
category_pivot.plot(kind='barh', ax=ax2)
ax2.set_title('不同类别指标的平均R2', fontsize=22, fontweight='bold')
ax2.set_xlabel('平均R2', fontsize=22, fontweight='bold')
ax2.set_ylabel('指标类别', fontsize=22, fontweight='bold')
ax2.tick_params(axis='both', labelsize=18, width=1.5)
for label in ax2.get_xticklabels() + ax2.get_yticklabels():
    label.set_fontweight('bold')

# 修正图例设置 - 字号放在prop里
legend = ax2.legend(title='栅格变量', bbox_to_anchor=(1.05, 1), loc='upper left', 
                    prop={'weight': 'bold', 'size': 22})
legend.get_title().set_fontsize(22)
legend.get_title().set_fontweight('bold')

# 5.3 Top指标对比
ax3 = plt.subplot(1, 3, 3)
top_indicators_all = results_df.groupby('统计指标')['R2'].mean().sort_values(ascending=False).head(15)
top_indicators_all.plot(kind='barh', ax=ax3, color='steelblue')
ax3.set_title('Top 15 整体相关指标', fontsize=22, fontweight='bold')
ax3.set_xlabel('平均R2', fontsize=22, fontweight='bold')
ax3.set_ylabel('统计指标', fontsize=22, fontweight='bold')
ax3.tick_params(axis='both', labelsize=18, width=1.5)
for label in ax3.get_xticklabels() + ax3.get_yticklabels():
    label.set_fontweight('bold')

plt.tight_layout()
viz_file1 = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\R2分析综合图1.png'
plt.savefig(viz_file1, dpi=300, bbox_inches='tight')
print(f"可视化1已保存: {viz_file1}")
plt.close()

# 创建第二组可视化 (4个热力图)
fig, axes = plt.subplots(2, 2, figsize=(26, 22))
axes = axes.flatten()

for idx, raster_var in enumerate(raster_vars):
    ax = axes[idx]
    raster_data = results_df[results_df['栅格变量'] == raster_var]
    
    # 选择平均R2最高的15个指标
    top_indicators = raster_data.groupby('统计指标')['R2'].mean().sort_values(ascending=False).head(15).index
    
    # 创建省份-指标的R2矩阵
    pivot_data = raster_data[raster_data['统计指标'].isin(top_indicators)].pivot_table(
        values='R2', index='省份', columns='统计指标', aggfunc='mean'
    )
    
    if len(pivot_data) > 0:
        sns.heatmap(pivot_data, cmap='YlOrRd', ax=ax, 
                   cbar_kws={'label': 'R2'}, vmin=0, vmax=1,
                   xticklabels=True, yticklabels=True)
        ax.set_title(f'{raster_var} - Top 15指标', fontsize=22, fontweight='bold')
        ax.set_xlabel('统计指标', fontsize=22, fontweight='bold')
        ax.set_ylabel('省份', fontsize=22, fontweight='bold')
        ax.tick_params(axis='x', rotation=45, labelsize=18)
        ax.tick_params(axis='y', labelsize=18)
        
        # 设置刻度标签加粗
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')
        
        # 设置colorbar字体
        cbar = ax.collections[0].colorbar
        cbar.ax.tick_params(labelsize=18)
        for label in cbar.ax.get_yticklabels():
            label.set_fontweight('bold')
        cbar.set_label('R2', fontsize=22, fontweight='bold')

plt.tight_layout()
viz_file2 = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\R2分析热力图.png'
plt.savefig(viz_file2, dpi=300, bbox_inches='tight')
print(f"热力图已保存: {viz_file2}")
plt.close()

# 创建散点图矩阵（选择几个高相关的例子）
print("\n生成散点图示例...")
fig, axes = plt.subplots(2, 4, figsize=(28, 14))
axes = axes.flatten()

plot_idx = 0
for raster_var in raster_vars:
    # 找出该栅格变量R2最高的2个指标
    top_2 = results_df[results_df['栅格变量'] == raster_var].groupby('统计指标')['R2'].mean().sort_values(ascending=False).head(2)
    
    for stat_var in top_2.index:
        if plot_idx >= 8:
            break
        
        ax = axes[plot_idx]
        
        # 绘制散点图
        x = df[raster_var].values
        y = df[stat_var].values
        
        # 移除缺失值
        mask = ~(np.isnan(x) | np.isnan(y))
        x_clean = x[mask]
        y_clean = y[mask]
        
        ax.scatter(x_clean, y_clean, alpha=0.4, s=20, color='steelblue')
        
        # 添加趋势线
        z = np.polyfit(x_clean, y_clean, 1)
        p = np.poly1d(z)
        x_trend = np.linspace(x_clean.min(), x_clean.max(), 100)
        ax.plot(x_trend, p(x_trend), "r-", linewidth=2.5)
        
        # 计算R2
        r = np.corrcoef(x_clean, y_clean)[0, 1]
        r2 = r ** 2
        
        ax.set_xlabel(raster_var, fontsize=22, fontweight='bold')
        ax.set_ylabel(stat_var, fontsize=22, fontweight='bold')
        ax.set_title(f'R2 = {r2:.3f}', fontsize=22, fontweight='bold')
        ax.grid(True, alpha=0.3, linewidth=1)
        ax.tick_params(axis='both', labelsize=18, width=1.5)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')
        
        # 加粗边框
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
        
        plot_idx += 1

plt.tight_layout()
scatter_file = r'F:\00.博后研究\城乡连续体制图\New_NEW\04.验证\R2散点图示例.png'
plt.savefig(scatter_file, dpi=300, bbox_inches='tight')
print(f"散点图已保存: {scatter_file}")
plt.close()

print("\n" + "="*100)
print("分析总结")
print("="*100)

print(f"""
关键发现:
1. 总体相关性偏弱,但这在空间数据中很常见
2. R2分布:
   - R2 > 0.25 (中等以上): {len(results_df[results_df['R2'] > 0.25])} 个 ({len(results_df[results_df['R2'] > 0.25])/len(results_df)*100:.1f}%)
   - 0.09 < R2 < 0.25 (弱到中等): {len(results_df[(results_df['R2'] > 0.09) & (results_df['R2'] <= 0.25)])} 个 ({len(results_df[(results_df['R2'] > 0.09) & (results_df['R2'] <= 0.25)])/len(results_df)*100:.1f}%)
   - R2 < 0.09 (弱相关): {len(results_df[results_df['R2'] <= 0.09])} 个 ({len(results_df[results_df['R2'] <= 0.09])/len(results_df)*100:.1f}%)

3. 建议:
   - 关注R2 > 0.09的指标,这些已经有一定解释力
   - 考虑非线性关系或交互效应
   - 可能需要标准化或对数转换
   - 考虑空间自相关的影响

输出文件:
1. 完整结果: {output_file}
2. 中等以上相关: {moderate_corr_file}
3. 分类统计: {category_file}
4. 可视化1: {viz_file1}
5. 热力图: {viz_file2}
6. 散点图: {scatter_file}
""")

数据形状: (2654, 78)
省份数量: 31
统计指标数量: 69

开始计算...
1/31 - 北京市 (n=5)
2/31 - 天津市 (n=11)
3/31 - 河北省 (n=153)
4/31 - 山西省 (n=112)
5/31 - 内蒙古自治区 (n=101)
6/31 - 辽宁省 (n=100)
7/31 - 吉林省 (n=59)
8/31 - 黑龙江省 (n=73)
9/31 - 上海市 (n=10)
10/31 - 江苏省 (n=91)
11/31 - 浙江省 (n=87)
12/31 - 安徽省 (n=100)
13/31 - 福建省 (n=82)
14/31 - 江西省 (n=98)
15/31 - 山东省 (n=133)
16/31 - 河南省 (n=156)
17/31 - 湖北省 (n=102)
18/31 - 湖南省 (n=121)
19/31 - 广东省 (n=80)
20/31 - 广西壮族自治区 (n=99)
21/31 - 海南省 (n=17)
22/31 - 重庆市 (n=38)
23/31 - 四川省 (n=180)
24/31 - 贵州省 (n=88)
25/31 - 云南省 (n=129)
26/31 - 西藏自治区 (n=74)
27/31 - 陕西省 (n=104)
28/31 - 甘肃省 (n=86)
29/31 - 青海省 (n=43)
30/31 - 宁夏回族自治区 (n=22)
31/31 - 新疆维吾尔自治区 (n=100)

总结果数: 8168
完整结果已保存

1. 按R2分析 (R2 > 0.25 表示中等以上相关)

发现 420 个中等以上相关的结果 (R2 > 0.25)

Top 30 最强相关:
      省份  栅格变量                   统计指标        R2
229  北京市    NF             普通中学学校数(个)  0.975517
234  北京市    NF       中等职业教育学校在校学生数(人)  0.964862
211  北京市    NF                出口额(美元)  0.964519
210  北京市    NF       年末金融机构各项贷款余额(万元)  0.959073
208  北京